In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)

### Hack window for playing with ideas

In [2]:
'''
fi = FileInfo('ext')
fi.textLineCount = 1644
plusCount = 29
minusCount = 0
n = plusCount * 1.0
print(n)
n = n / (plusCount + minusCount)
print(n)
print(fi.textLineCount)
n = n * (1.0 * fi.textLineCount)
print(n)
fi.insertCount = int(fi.textLineCount * ((plusCount * 1.0) / (plusCount + minusCount)))
fi.deleteCount = fi.textLineCount - fi.insertCount
print(fi.insertCount, ',', fi.deleteCount)
'''
0

0

In [3]:
def introspect(obj):
  for func in [type, id, dir, vars, callable]:
        print("%s(%s):\t\t%s" % (func.__name__, introspect.__code__.co_varnames[0], func(obj)))


### Main Entry Point

In [ ]:
import sys
project_root_path = '../../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_log_parser
import hashlib
from pathlib import Path
import mariadb
import json
import hashlib
from datetime import datetime as datingdays

mydb = mariadb.connect(
    host='localhost',
    user='w3hacknet',
    password='aSecret42',
    database='w3hacknet',
    autocommit=True
)

cursor = mydb.cursor()

def insert_result(array, owner, repo_name):
    '''
    CREATE PROCEDURE w3hacknet.InsertCommit(owner_id varchar(128), repo_name varchar(128), commit_id char(40), author_hash char(32), author_alias varchar(128), date datetime, orig_timezone varchar(16))
    '''
    for n in array:
        cursor.callproc('w3hacknet.InsertCommit', 
                        (owner,
                         repo_name, 
                         n['commit'], 
                         hashlib.md5(n['Author'].encode('utf-8')).hexdigest(),
                         n['Author'],
                         datingdays.fromisoformat(n['Date']),
                         n['orig_timezone'],
                         json.dumps(n['fileTypes']) ) )
    print('Processed',len(array),'records for', owner, repo_name)
                         
                         

fileList = []
commit_log_parser.addFiles(fileList, '/home/matt/Projects/Web3HackerNetwork/sandbox/notebooks/matt/results');
print('Found '+str(len(fileList))+' files');
for n in fileList:
    if n.fileName == 'commit_stat_log.json':
        print('Processing', n.fullyQualified)
        p = Path(n.fullyQualified)
        print('https://github.com/'+p.parent.parent.name+'/'+p.parent.name)
        with open(n.fullyQualified, 'r') as r:
            insert_result(json.load(r), p.parent.parent.name, p.parent.name)
        #introspect(p.parent)
'''
root = '/home/matt/Projects/Web3HackerNetwork/data/samples'
log = open(root+'/commitScan.log', 'w')
for fileClass in fileList:
    linecount = 0
    
    if (fileClass.fileName != 'commit-stat.log'):
        #print('Skipping '+fileClass.fileName)
        a = 0
    else:
        fq = fileClass.fullyQualified
        print('Processing ',fq)
        tsvFileName = str(fq)+'.json'
        result = open(tsvFileName, 'w')

        reqSet = StatRequirementSet()

        with open(str(fileClass.fullyQualified), 'r') as file:
            log.write('Reading file: '+fileClass.fullyQualified+'\n')

            for line in file:
                linecount += 1
                reqSet.testline(line);
            file.close()
            
        result.write(json.dumps(reqSet.resultArray, indent=2))
        reqSet.resultArray.clear()
        nBytes = result.tell()
        result.close();
        if (nBytes < 1):
            os.remove(tsvFileName)


        print(str(fileClass.fileName)+' : '+str(linecount)+' lines')
'''
print('All done!')


Found 206 files
Processing /home/matt/Projects/Web3HackerNetwork/sandbox/notebooks/matt/results/polkadot-js/api/commit_stat_log.json
https://github.com/polkadot-js/api
Processed 6394 records for polkadot-js api
Processing /home/matt/Projects/Web3HackerNetwork/sandbox/notebooks/matt/results/ApeWorX/ape/commit_stat_log.json
https://github.com/ApeWorX/ape
Processed 370 records for ApeWorX ape
Processing /home/matt/Projects/Web3HackerNetwork/sandbox/notebooks/matt/results/mozilla/glam/commit_stat_log.json
https://github.com/mozilla/glam
Processed 961 records for mozilla glam
Processing /home/matt/Projects/Web3HackerNetwork/sandbox/notebooks/matt/results/mozilla/zamboni/commit_stat_log.json
https://github.com/mozilla/zamboni
Processed 21570 records for mozilla zamboni
Processing /home/matt/Projects/Web3HackerNetwork/sandbox/notebooks/matt/results/Biohazard1976/pip/commit_stat_log.json
https://github.com/Biohazard1976/pip


In [14]:
    rslt = mydb.commit()


### Testing time-zone aware date comparisons

In [ ]:
from datetime import datetime as datingdays
from pytz import timezone
last_date = datingdays.fromisoformat('1972-12-26T03:23:01.123456-07:00')
nowness = datingdays.now(timezone('US/Arizona'))
print(nowness - last_date)


### Clone/Pull all of the repositories listed in input file

In [ ]:
import os
from git import Repo, Git

with open('./new_repo.log', 'r') as r:
    for l in r.readlines():
        key = l[:-1] #Strip off carriage return
        s = key.split('/')
        owner = s[0]
        repo_name = s[1]
        full_name = key
        print('Processing', owner, repo_name)
        repo_base_dir = './repos'
        repo_path = repo_base_dir+'/'+key

        if (os.path.isdir(repo_base_dir) == False):
            print('######### Cannot find '+repo_base_dir+'  Creating it!')
            os.makedirs(repo_base_dir)
        if (os.path.isdir(repo_base_dir+"/"+owner) == False):
            os.makedirs(repo_base_dir+"/"+owner)
        url = 'https://github.com/'+owner+'/'+repo_name+'.git'
        if (os.path.isdir(repo_path) == False):
            Repo.clone_from(url, repo_path)
        else:
            rp = Repo(repo_path)
            remote = rp.remote()
            remote.pull()

### Better way of running git and gather statistics

In [ ]:
import git
import json
from datetime import datetime as datingdays
from pytz import timezone
from pydriller import Repository
import sys
project_root_path = '../../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_log_parser
from os import listdir
from os.path import isfile, join, isdir

last_date = datingdays.fromisoformat('1972-12-26T03:23:01.123456-07:00')

def introspect(obj):
  for func in [type, id, dir, vars, callable]:
        print("%s(%s):\t\t%s" % (func.__name__, introspect.__code__.co_varnames[0], func(obj)))

class CommitStats():
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=2)    
    def __init__(self):
        self.commit = ''
        self.commit_timestamp = ''
        self.author_name = ''
        self.author_email = ''
        self.file_stats_dic = {}
    def determine_file_type(self, file_name):
        retVal = 'blank'
        if file_name is not None and len(file_name) > 0:
            s = file_name.split('.')
            if len(s) > 1:
                retVal = s[len(s)-1]
        return retVal
    def process_commit(self, c):
        statPerf = 0
        base = datingdays.now().timestamp()
        if c.author is None or c.author.name is None or c.author.email is None:
            print('No author found in commit: ', c.hexsha)
            if c.parents is not None and len(c.parents) > 0:
                print('Trying recursive for ', c.parents[0].hexsha)
                process_commit(c.parents[0])
        else:
            self.commit = c.hash
            self.author_name = c.author.name
            self.author_email = c.author.email
            self.commit_timestamp = datingdays.isoformat(c.committer_date)
            for stat in c.modified_files:
                file_type = self.determine_file_type(stat.filename)
                if file_type not in self.file_stats_dic.keys():
                    self.file_stats_dic[file_type] = {}
                    self.file_stats_dic[file_type]['file_count'] = 0
                    self.file_stats_dic[file_type]['insert_count'] = 0
                    self.file_stats_dic[file_type]['delete_count'] = 0
                self.file_stats_dic[file_type]['file_count'] += 1
                i = stat.added_lines
                d = stat.deleted_lines
                if i == 0 and d == 0:
                    pass
#                    print('Working dir? '+repo.working_dir)
                else:
                    self.file_stats_dic[file_type]['insert_count'] += i
                    self.file_stats_dic[file_type]['delete_count'] += d
                

file_dic = {}
user_map = {}
print(datingdays.now())
master_map = {}
with open('./new_repo.log', 'r') as r:
    for l in r.readlines():
        base_map = {}
        key = l[:-1] #Strip off carriage return
        s = key.split('/')
        owner = s[0]
        repo_name = s[1]
        full_name = key
        repo_base_dir = './repos'
        repo_path = repo_base_dir+'/'+key
        
        cnt = 0
        for commit in Repository(repo_path).traverse_commits():
            cnt += 1
            if cnt % 100 == 0:
                print(cnt,'Processing', owner, repo_name, commit.author.name, commit.author_date, commit.hash)
            obj = CommitStats()
            obj.process_commit(commit)
            base_map[obj.commit] = obj
        print(cnt, 'commits processed total')
        with open(repo_base_dir+'/commit_stat_log.json', 'w') as w:
            w.write(json.dumps(base_map, default=lambda o: o.__dict__, sort_keys=True, indent=2))
        master_map[key] = base_map
        
with open('./master_commit_stat_log.json', 'w') as w:
    w.write(json.dumps(master_map, default=lambda o: o.__dict__, sort_keys=True, indent=2))
    
'''    
    d = c.committed_datetime
    print(d)
    if d > last_date:
        last_date = d;
#    print(c.stats.total)
    for f in c.stats.files:
        stat = c.stats.files[f]
        print(f, stat['insertions'], stat['deletions'], stat['lines'])
    
print('Latest date: ', last_date)  
'''
print('done')

### Multi-threaded status logger (experiment)

In [ ]:
import time
import threading
import logging
class Thing():
    def __init__(self):
        format = "%(asctime)s: %(message)s"
        logging.basicConfig(format=format, level=logging.INFO,
                            datefmt="%H:%M:%S")        
        self.thread = threading.Thread(target=self.monitor, daemon=True)
        self.thread.start()
        self.running = True
    def stop(self):
        self.running = false
        self.thread.interrupt()
    def monitor(self):
        print('Made it here')
        while self.running:
            time.sleep(5)
            logging.info("Still here monitoring")
            
t = Thing()
print("Waiting around to see if background thread works")
print(t.thread)
time.sleep(30)
t.running = False
t.join()
print("Leaving...")

